In [1]:
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.2.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 261 kB in 2s (109 kB/s)
Reading package lis

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-11-11 04:00:51--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.08s   

2022-11-11 04:00:51 (10.3 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
# create spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("big-data-challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [6]:
from pyspark import SparkFiles
# Load file

url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Ebook_Purchase_v1_00.tsv.gz" 
# get data from amazon S3 and  unzip extract files 

spark.sparkContext.addFile(url)

ebooks_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Digital_Ebook_Purchase_v1_00.tsv.gz"), header=True, inferSchema=True, sep='\t', timestampFormat="mm/dd/yy")

ebooks_df.show(10)

NameError: ignored

In [28]:
ebooks_df.show(10)
ebooks_df.count()


+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   35471030|R1C5OK9AND7PRP|B00AHK07V0|     175130663|Hunter's Moon (A ...|Digital_Ebook_Pur...|          5|            0|          0|   N|                Y|Shugak is like a ...|This is the most ...| 2015-08-31|
|         US|   26579324|R333RNBQMPVUFT|B014085WTQ|     859232728|Flying Toward For...|Digital_Ebook_Pur

12518702

In [9]:

url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Music_v1_00.tsv.gz" 
# get data from amazon S3 and  unzip extract files 

spark.sparkContext.addFile(url)

music_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Music_v1_00.tsv.gz"), header=True, inferSchema=True, sep='\t', timestampFormat="mm/dd/yy")

music_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   10140119|R3LI5TRP3YIDQL|B00TXH4OLC|     384427924|Whatever's for Us...|           Music|          5|            0|          0|   N|                Y|          Five Stars|Love this CD alon...| 2015-08-31|
|         US|   27664622|R3LGC3EKEG84PX|B00B6QXN6U|     831769051|Same Trailer Diff...|           Music|          5|    

In [10]:
music_df.count()

4751577

In [ ]:
#tranforming data to match the columns in the table where the data will be imported
#data will be imported into database that is running in AWS
# use user_payment_df = user_payment_df.withColumnRenamed("id","billing_id") to rename columns 


In [14]:
# Drop null values, by default is any which means will drop any rows which has a NA
ebooks_df = ebooks_df.dropna(how='any')
ebooks_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   35471030|R1C5OK9AND7PRP|B00AHK07V0|     175130663|Hunter's Moon (A ...|Digital_Ebook_Pur...|          5|            0|          0|   N|                Y|Shugak is like a ...|This is the most ...| 2015-08-31|
|         US|   26579324|R333RNBQMPVUFT|B014085WTQ|     859232728|Flying Toward For...|Digital_Ebook_Pur

In [15]:
ebooks_df.count()

12518702

In [25]:
#create dataframe that corresponds to customers table  with counts of records per user
ebooks_df_customers = ebooks_df.groupby("customer_id").agg({"customer_id": "count"})
ebooks_df_customers.show(10)
ebooks_df.dtypes

+-----------+------------------+
|customer_id|count(customer_id)|
+-----------+------------------+
|   41341751|                18|
|   14582882|                 1|
|   15158447|                 7|
|    3070853|                16|
|   26248773|               511|
|   50058978|                 1|
|   16667845|                 3|
|   14217455|                 1|
|   14754839|                 6|
|   35311351|                 7|
+-----------+------------------+
only showing top 10 rows



[('marketplace', 'string'),
 ('customer_id', 'int'),
 ('review_id', 'string'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('product_title', 'string'),
 ('product_category', 'string'),
 ('star_rating', 'string'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string'),
 ('verified_purchase', 'string'),
 ('review_headline', 'string'),
 ('review_body', 'string'),
 ('review_date', 'string')]

In [24]:
##create dataframe that corresponds to columns in Products to load into data
product_df = ebooks_df.select(["product_id","product_title"])
product_df.show(10)
product_df.dtypes

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00AHK07V0|Hunter's Moon (A ...|
|B014085WTQ|Flying Toward For...|
|B0113MFPN0|Falling For A Rea...|
|B00T8RIK7G|   The Good Neighbor|
|B00EMXBDMA|The Martian: A Novel|
|B00GL3OJIY|The Painter: A novel|
|B00RW30QE8|        METEOR STORM|
|B0101AE85Q|Diary of a Crazy ...|
|B00IO7QAI2|Unlocking Potenti...|
|B00PM995TG|The Einstein Prop...|
+----------+--------------------+
only showing top 10 rows



[('product_id', 'string'), ('product_title', 'string')]

In [23]:
#create dataframe that corresponds to columns in vine_table to load into data
vine_df = ebooks_df.select(["review_id","star_rating","helpful_votes","total_votes","vine"])
vine_df.show(10)
vine_df.dtypes

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1C5OK9AND7PRP|          5|            0|          0|   N|
|R333RNBQMPVUFT|          5|            0|          0|   N|
|R2A2K0GTNSKXM1|          5|            0|          0|   N|
|R365LAQ9REV876|          3|            0|          0|   N|
|R294OF3SNH6SWZ|          4|            0|          0|   N|
|R2OMRH58WYEEP9|          5|            0|          0|   N|
| RBFXYUJZIH5JR|          5|            0|          0|   N|
|R2K2RSBNV42HU4|          5|            0|          0|   N|
|R2CLRFFJ5HJSU3|          4|            0|          0|   N|
| RS1O7V45AADDO|          4|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 10 rows



[('review_id', 'string'),
 ('star_rating', 'string'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string')]

In [27]:
review_id_df = ebooks_df.select(["review_id","customer_id","product_id","product_parent","review_date"])
review_id_df.show(10)
review_id_df.dtypes

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1C5OK9AND7PRP|   35471030|B00AHK07V0|     175130663| 2015-08-31|
|R333RNBQMPVUFT|   26579324|B014085WTQ|     859232728| 2015-08-31|
|R2A2K0GTNSKXM1|    9152364|B0113MFPN0|     411472516| 2015-08-31|
|R365LAQ9REV876|   52087605|B00T8RIK7G|     353828021| 2015-08-31|
|R294OF3SNH6SWZ|   46454660|B00EMXBDMA|     803172158| 2015-08-31|
|R2OMRH58WYEEP9|   33168429|B00GL3OJIY|     325772386| 2015-08-31|
| RBFXYUJZIH5JR|   18546798|B00RW30QE8|     236508296| 2015-08-31|
|R2K2RSBNV42HU4|   47463409|B0101AE85Q|     924272105| 2015-08-31|
|R2CLRFFJ5HJSU3|   11767467|B00IO7QAI2|     393059633| 2015-08-31|
| RS1O7V45AADDO|   14241457|B00PM995TG|     529331494| 2015-08-31|
+--------------+-----------+----------+--------------+-----------+
only showing top 10 rows



[('review_id', 'string'),
 ('customer_id', 'int'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('review_date', 'string')]